# ・ライブラリのインポート

In [13]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn import model_selection 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

seed = 0

# ・データの読み込み

In [14]:
train = pd.read_csv('/Users/markun/SIGNATE/train.csv')
test = pd.read_csv('/Users/markun/SIGNATE/test.csv')

In [15]:
test

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,398,0,126,80,0,0,40.096264,0.822517,21
1,3833,3,88,60,20,0,39.810590,0.204331,22
2,4836,3,114,76,0,0,33.198760,0.521011,21
3,4572,1,146,74,0,0,26.890259,0.504950,38
4,636,1,123,90,26,140,40.270088,0.800513,28
...,...,...,...,...,...,...,...,...,...
1995,3138,4,150,60,0,0,39.385785,0.175051,26
1996,191,6,153,88,0,0,38.614204,0.509367,28
1997,3294,7,106,78,0,0,51.678147,0.728404,29
1998,3073,2,101,70,0,0,40.271989,0.316558,26


In [16]:
train

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,200,9,125,74,0,0,28.536910,0.444902,45,1
1,3832,4,109,80,0,0,28.047673,0.238243,22,0
2,4927,4,88,78,39,0,52.371341,0.279471,26,0
3,4088,9,125,74,0,0,40.062688,0.203922,45,0
4,3644,5,107,78,44,284,52.935068,0.284959,45,1
...,...,...,...,...,...,...,...,...,...,...
2995,4931,4,88,74,17,0,33.848723,0.171073,23,0
2996,3264,0,144,88,0,0,26.846832,0.259957,21,1
2997,1653,6,117,96,36,0,28.101646,0.716126,22,1
2998,2607,2,113,74,0,0,33.079021,0.266179,38,1


# ・目的変数を除外

In [17]:
train_x = train.drop(['Outcome'], axis = 1)
train_x 

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,200,9,125,74,0,0,28.536910,0.444902,45
1,3832,4,109,80,0,0,28.047673,0.238243,22
2,4927,4,88,78,39,0,52.371341,0.279471,26
3,4088,9,125,74,0,0,40.062688,0.203922,45
4,3644,5,107,78,44,284,52.935068,0.284959,45
...,...,...,...,...,...,...,...,...,...
2995,4931,4,88,74,17,0,33.848723,0.171073,23
2996,3264,0,144,88,0,0,26.846832,0.259957,21
2997,1653,6,117,96,36,0,28.101646,0.716126,22
2998,2607,2,113,74,0,0,33.079021,0.266179,38


# ・目的変数

In [18]:
train_y = train['Outcome']
train_y

0       1
1       0
2       0
3       0
4       1
       ..
2995    0
2996    1
2997    1
2998    1
2999    0
Name: Outcome, Length: 3000, dtype: int64

# ・訓練データを訓練用と検証用に分割する

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y,
                                                        test_size=0.2,
                                                        shuffle=True,
                                                        random_state=42,
                                                        stratify=train_y)
kfold = model_selection.KFold(n_splits = 5)
scores = {}

In [20]:
# ロジスティック回帰
lr_clf = LogisticRegression(solver='lbfgs', max_iter=10000)
lr_clf.fit(X_train, y_train)
results = model_selection.cross_val_score(lr_clf, X_valid, y_valid, cv = kfold) 
scores[('1.Logistic_regression', 'train_score')] = results.mean()
scores[('1.Logistic_regression', 'test_score')] = lr_clf.score(X_valid, y_valid)

# 決定木
dtc_clf = DecisionTreeClassifier(random_state=seed)
dtc_clf.fit(X_train, y_train)
results = model_selection.cross_val_score(dtc_clf, X_valid, y_valid, cv = kfold) 
scores[('2.decision_tree', 'train_score')] = results.mean()
scores[('2.decision_tree', 'test_score')] = dtc_clf.score(X_valid, y_valid)

# サポートベクターマシン（SVM）
svm_clf = SVC(probability=True, random_state=seed)
svm_clf.fit(X_train, y_train)
results = model_selection.cross_val_score(svm_clf, X_valid, y_valid, cv = kfold) 
scores[('3.SVM', 'train_score')] = results.mean()
scores[('3.SVM', 'test_score')] = svm_clf.score(X_valid, y_valid)

# AdaBoost
adb_clf = AdaBoostClassifier(n_estimators=100, random_state=seed)
adb_clf.fit(X_train, y_train)
results = model_selection.cross_val_score(adb_clf, X_valid, y_valid, cv = kfold) 
scores[('4.AdaBoost', 'train_score')] = results.mean()
scores[('4.AdaBoost', 'test_score')] = adb_clf.score(X_valid, y_valid)

# GradientBoosting (GBM)
gbm_clf = GradientBoostingClassifier(random_state=seed)
gbm_clf.fit(X_train, y_train)
results = model_selection.cross_val_score(gbm_clf, X_valid, y_valid, cv = kfold) 
scores[('5.GBM', 'train_score')] = results.mean()
scores[('5.GBM', 'test_score')] = gbm_clf.score(X_valid, y_valid)

# ランダムフォレスト
rfc_clf = RandomForestClassifier(max_depth=5, random_state=seed)
rfc_clf.fit(X_train, y_train)
results = model_selection.cross_val_score(rfc_clf, X_valid, y_valid, cv = kfold) 
scores[('6.Random Forest', 'train_score')] = results.mean()
scores[('6.Random Forest', 'test_score')] = rfc_clf.score(X_valid, y_valid)

# xgboost
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)
results = model_selection.cross_val_score(xgb_clf, X_valid, y_valid, cv = kfold) 
scores[('7.xgboost', 'train_score')] = results.mean()
scores[('7.xgboost', 'test_score')] = xgb_clf.score(X_valid, y_valid)


# バギングアンサンブル
estimators=[('lr', lr_clf), ('dtc', dtc_clf), ('svc', svm_clf),('xgb',xgb_clf),('rfc',rfc_clf),('gbm',gbm_clf),('adb',adb_clf)]
vote_clf=VotingClassifier(estimators=estimators, voting='hard')
pred = vote_clf.fit(X_train, y_train).predict(test)
results = model_selection.cross_val_score(vote_clf, X_valid, y_valid, cv = kfold) 
scores[('8.Max_voting', 'train_score')] = results.mean()
scores[('8.Max_voting', 'test_score')] = vote_clf.score(X_valid, y_valid)

pd.Series(scores).unstack()

,test_score,train_score
1.Logistic_regression,0.770000,0.765000
2.decision_tree,0.723333,0.693333
3.SVM,0.761667,0.761667
4.AdaBoost,0.800000,0.746667
5.GBM,0.801667,0.766667
6.Random Forest,0.786667,0.780000
7.xgboost,0.785000,0.761667
8.Max_voting,0.803333,0.776667


#  ・テストデータに対する予測

In [21]:
pred

array([0, 0, 0, ..., 0, 0, 0])

# ・提出用ファイルの作成

In [22]:
submission_ensemble = pd.DataFrame({
    'index':test['index'],
    'Outcome':pred, 
})
submission_ensemble.to_csv('/Users/markun/SIGNATE/submission_ensemble.csv', header = False, index = False )

In [23]:
submission_ensemble.head()

,index,Outcome
0,398,0
1,3833,0
2,4836,0
3,4572,0
4,636,0
